In [22]:
from os import remove
from py7zr import SevenZipFile
import pandas as pd
import wget
import numpy as np

## Dicionário de Dados
Admitidos/Desligados: Contratação (1), Demissão(2)

Sexo: Masculino (1), Feminino (2)

Classificação de Setores CNAE 2.0: https://docs.google.com/spreadsheets/d/1bMFKpREb4YlW2ZzW1WVLdMN59ol-iLw5/export?format=xlsx

Códigos IBGE por Estado: https://www.oobj.com.br/bc/article/quais-os-c%C3%B3digos-de-cada-uf-no-brasil-465.html

In [23]:
excel = pd.ExcelFile("data/dicionario_dados.xlsx")
get_dict =  lambda x: pd.read_excel(excel, sheet_name=x)

data_dict = {
    sheet: {row[1]: row[2] for row in 
     get_dict(sheet).itertuples()}
    for sheet in excel.sheet_names[1:]
}

### Grandes grupos

In [8]:
big_group = {
	'Administração Pública, Defesa e Seguridade Social': 'Serviços',
	'Agricultura, Pecuária, Produção Florestal, Pesca e AqÜIcultura': 'Agropecuária',
	'Alojamento e Alimentação': 'Serviços',
	'Artes, Cultura, Esporte e Recreação': 'Serviços',
	'Atividades Administrativas e Serviços Complementares': 'Serviços',
	'Atividades Financeiras, de Seguros e Serviços Relacionados': 'Serviços',
	'Atividades Imobiliárias': 'Serviços',
	'Atividades Profissionais, Científicas e Técnicas': 'Serviços',
	'Comércio, Reparação de Veículos Automotores e Motocicletas': 'Comércio',
	'Construção': 'Construção',
	'Educação': 'Serviços',
	'Eletricidade e Gás': 'Indústria',
	'Indústrias Extrativas': 'Indústria',
	'Indústrias de Transformação': 'Indústria',
	'Informação e Comunicação': 'Serviços',
	'Não identificado': 'Não Identificado',
	'Organismos Internacionais e Outras Instituições Extraterritoriais': 'Serviços',
	'Outras Atividades de Serviços': 'Serviços',
	'Saúde Humana e Serviços Sociais': 'Serviços',
	'Serviços Domésticos': 'Serviços',
	'Transporte, Armazenagem e Correio': 'Serviços',
	'Água, Esgoto, Atividades de Gestão de Resíduos e Descontaminação':
	'Indústria'
}

## Leitura e processamento

In [24]:
url = lambda year, month: f"ftp://ftp.mtps.gov.br/pdet/microdados/NOVO CAGED/{year}/{year}{month:02d}/CAGEDMOV{year}{month:02d}.7z"

In [25]:
#Baixando e extraindo microdados
dfs = []
start_year = 2023
start_month = 1
dates = []
# dates = data["competênciamov"].unique()


for year in range(start_year, 2025):
    for month in range(start_month, 13):
        if f"{year}-{month:02d}-01" in dates:
            continue
        try:
            print(f"{month:02d}/{year}")
            wget.download(url(year, month), 'caged.7z')
            archive = SevenZipFile('caged.7z', mode = 'r')
            print('Microdados baixados com sucesso, passando para leitura')
            for name, fd in archive.read(name for name in archive.getnames() if name.endswith(".txt")).items():
                caged_raw = pd.read_csv(fd, delimiter=";", decimal=",")
                caged_raw = caged_raw.loc[caged_raw["uf"] == 25, :].reset_index(drop=True)
                for col in caged_raw.columns:
                    if col in data_dict:
                        caged_raw[f"{col}_cod"] = caged_raw[col]
                        caged_raw[col] = caged_raw[col].apply(lambda x: data_dict[col][x] 
                                                              if x in data_dict[col] else np.nan)

            dfs.append(caged_raw)
            archive.close()
            remove('caged.7z')
            print('Leitura concluida com sucesso')
        except:
            print('Os microdados do mês selecionado ainda não estão disponíveis')
            break

01/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
02/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
03/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
04/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
05/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
06/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
07/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
08/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
09/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
10/2023
Microdados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
11/2023
Microdados baixados com sucesso, passando para leitura
Leitura

In [11]:
new_data = pd.concat(dfs)

In [26]:
data = pd.concat(dfs)

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480152 entries, 0 to 37508
Data columns (total 49 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   competênciamov              480152 non-null  int64  
 1   região                      480152 non-null  object 
 2   uf                          480152 non-null  object 
 3   município                   480152 non-null  object 
 4   seção                       480152 non-null  object 
 5   subclasse                   480133 non-null  object 
 6   saldomovimentação           480152 non-null  int64  
 7   cbo2002ocupação             480114 non-null  object 
 8   categoria                   480152 non-null  object 
 9   graudeinstrução             480152 non-null  object 
 10  idade                       480142 non-null  float64
 11  horascontratuais            480053 non-null  object 
 12  raçacor                     480152 non-null  object 
 13  sexo           

In [28]:
data.head(2)

,competênciamov,região,uf,município,seção,subclasse,saldomovimentação,cbo2002ocupação,categoria,graudeinstrução,...,tipoestabelecimento_cod,tipomovimentação_cod,tipodedeficiência_cod,indtrabintermitente_cod,indtrabparcial_cod,tamestabjan_cod,indicadoraprendiz_cod,origemdainformação_cod,indicadordeforadoprazo_cod,unidadesaláriocódigo_cod
0,202301,Nordeste,Paraíba,Pb-Joao Pessoa,"Comércio, Reparação de Veículos Automotores e ...",Comércio Varejista Especializado de Equipament...,-1,Promotor de Vendas Especializado,"Empregado - Geral, inclusive o empregado públi...",Médio Completo,...,1,31,0,0,0,4,0,1,0,5
1,202301,Nordeste,Paraíba,Pb-Campina Grande,Atividades Administrativas e Serviços Compleme...,Limpeza em Prédios e em Domicílios,-1,Faxineiro (Desativado em 2010),"Empregado - Geral, inclusive o empregado públi...",Médio Completo,...,1,40,0,0,0,10,0,1,0,5


In [20]:
data = pd.concat([new_data, data])

In [29]:
data.to_csv("data/caged_pb.csv")

In [30]:
data = pd.read_csv("data/caged_pb.csv", index_col=0)
data["competênciamov"] = pd.to_datetime(data["competênciamov"], format="%Y%m")

In [31]:
data.head(1)

,competênciamov,região,uf,município,seção,subclasse,saldomovimentação,cbo2002ocupação,categoria,graudeinstrução,...,tipoestabelecimento_cod,tipomovimentação_cod,tipodedeficiência_cod,indtrabintermitente_cod,indtrabparcial_cod,tamestabjan_cod,indicadoraprendiz_cod,origemdainformação_cod,indicadordeforadoprazo_cod,unidadesaláriocódigo_cod
0,2023-01-01,Nordeste,Paraíba,Pb-Joao Pessoa,"Comércio, Reparação de Veículos Automotores e ...",Comércio Varejista Especializado de Equipament...,-1,Promotor de Vendas Especializado,"Empregado - Geral, inclusive o empregado públi...",Médio Completo,...,1,31,0,0,0,4,0,1,0,5


In [ ]:
data.to_csv("data/caged_pb.csv")

In [32]:
data.groupby(["competênciamov", "saldomovimentação"], 
             as_index=False).count()[["competênciamov", "saldomovimentação", "idade"]]

,competênciamov,saldomovimentação,idade
0,2023-01-01,-1,16251
1,2023-01-01,1,14528
2,2023-02-01,-1,14485
3,2023-02-01,1,15000
4,2023-03-01,-1,18040
5,2023-03-01,1,17226
6,2023-04-01,-1,17140
7,2023-04-01,1,13960
8,2023-05-01,-1,14366
9,2023-05-01,1,17235


In [35]:
data = pd.read_csv("data/caged_pb.csv", index_col=0, low_memory=False)
data["competênciamov"] = data["competênciamov"].apply(lambda x: f"{str(x)[:4]}-{str(x)[4:]}-01" if "-" not in str(x) else str(x))

g = data.groupby(["competênciamov", "município", "seção", "saldomovimentação", "sexo"], as_index=False).agg({
    "graudeinstrução": "count",
})

g.to_csv("data/caged_agregado.csv")

In [37]:
g.head(1)

,competênciamov,município,seção,saldomovimentação,sexo,graudeinstrução
0,2023-01-01,Pb-Agua Branca,"Comércio, Reparação de Veículos Automotores e ...",-1,Homem,1


In [38]:
g = pd.read_csv("data/caged_agregado22.csv", index_col=0)
g["competênciamov"] = pd.to_datetime(g["competênciamov"])

In [39]:
g.head(1)

,competênciamov,município,seção,saldomovimentação,sexo,graudeinstrução
0,2023-01-01,Pb-Agua Branca,"Comércio, Reparação de Veículos Automotores e ...",-1,Homem,1


In [40]:
g2 = g.astype({"competênciamov": str}).groupby(["competênciamov",
                                                            "saldomovimentação"], as_index=False).sum()
g2["graudeinstrução"] = g2["graudeinstrução"] * g2["saldomovimentação"]

g2 = g2.groupby("competênciamov").sum()


In [41]:
g2["graudeinstrução"] = g2["graudeinstrução"] * g2["saldomovimentação"]

g2 = g2.groupby("competênciamov").sum()


In [42]:
g2 = g.astype({"competênciamov": str}).groupby(["competênciamov",
                                                            "saldomovimentação"], as_index=False).sum()
g2["graudeinstrução"] = g2["graudeinstrução"] * g2["saldomovimentação"]
g2.groupby("competênciamov").sum()["graudeinstrução"]


competênciamov
2023-01-01   -1723
2023-02-01     514
2023-03-01    -815
2023-04-01   -3181
2023-05-01    2868
2023-06-01    -223
2023-07-01    3477
2023-08-01    8782
2023-09-01    4193
2023-10-01    3773
2023-11-01    3657
2023-12-01   -1504
2024-01-01     332
2024-02-01      -9
2024-03-01     263
Name: graudeinstrução, dtype: int64

In [43]:
g2.groupby("competênciamov").sum()["graudeinstrução"].to_dict()

{'2023-01-01': -1723,
 '2023-02-01': 514,
 '2023-03-01': -815,
 '2023-04-01': -3181,
 '2023-05-01': 2868,
 '2023-06-01': -223,
 '2023-07-01': 3477,
 '2023-08-01': 8782,
 '2023-09-01': 4193,
 '2023-10-01': 3773,
 '2023-11-01': 3657,
 '2023-12-01': -1504,
 '2024-01-01': 332,
 '2024-02-01': -9,
 '2024-03-01': 263}

In [52]:
data = pd.read_csv("data/caged_agregado.csv", index_col=0)
# data["competênciamov"] = pd.to_datetime(data["competênciamov"])
data["Grupo"] = data["seção"].apply(lambda x: big_group[x] if x in big_group else np.nan)

sectors_stats = data.groupby([
     "competênciamov", "saldomovimentação","Grupo"]).sum().to_dict()["graudeinstrução"]

# # final_stats = sectors_stats.groupby(["saldomovimentação", "Grupo"]).sum()["graudeinstrução"].to_dict()


In [50]:
data.head(1)

,competênciamov,município,seção,saldomovimentação,sexo,graudeinstrução,Grupo
0,2023-01-01,Pb-Agua Branca,"Comércio, Reparação de Veículos Automotores e ...",-1,Homem,1,Comércio


In [53]:
sectors_stats

{('2023-01-01', -1, 'Agropecuária'): 617,
 ('2023-01-01', -1, 'Comércio'): 4399,
 ('2023-01-01', -1, 'Construção'): 2179,
 ('2023-01-01', -1, 'Indústria'): 2835,
 ('2023-01-01', -1, 'Serviços'): 6221,
 ('2023-01-01', 1, 'Agropecuária'): 290,
 ('2023-01-01', 1, 'Comércio'): 3858,
 ('2023-01-01', 1, 'Construção'): 2417,
 ('2023-01-01', 1, 'Indústria'): 1427,
 ('2023-01-01', 1, 'Serviços'): 6536,
 ('2023-02-01', -1, 'Agropecuária'): 439,
 ('2023-02-01', -1, 'Comércio'): 3835,
 ('2023-02-01', -1, 'Construção'): 2373,
 ('2023-02-01', -1, 'Indústria'): 2211,
 ('2023-02-01', -1, 'Serviços'): 5628,
 ('2023-02-01', 1, 'Agropecuária'): 160,
 ('2023-02-01', 1, 'Comércio'): 3695,
 ('2023-02-01', 1, 'Construção'): 2897,
 ('2023-02-01', 1, 'Indústria'): 1840,
 ('2023-02-01', 1, 'Serviços'): 6408,
 ('2023-03-01', -1, 'Agropecuária'): 1196,
 ('2023-03-01', -1, 'Comércio'): 4017,
 ('2023-03-01', -1, 'Construção'): 2563,
 ('2023-03-01', -1, 'Indústria'): 4067,
 ('2023-03-01', -1, 'Serviços'): 6198,
 ('2

In [51]:
sectors_stats

{(-1, 'Agropecuária'): 1123,
 (-1, 'Comércio'): 4271,
 (-1, 'Construção'): 3009,
 (-1, 'Indústria'): 3209,
 (-1, 'Serviços'): 6264,
 (1, 'Agropecuária'): 231,
 (1, 'Comércio'): 4160,
 (1, 'Construção'): 3662,
 (1, 'Indústria'): 1911,
 (1, 'Serviços'): 7903}

In [75]:
import pandas as pd
import numpy as np

data = pd.read_csv("data/caged_agregado.csv", index_col=0)
# data["competênciamov"] = pd.to_datetime(data["competênciamov"])
data["Grupo"] = data["seção"].apply(lambda x: big_group[x] if x in big_group else np.nan)

sectors_stats = data.groupby([
     "competênciamov", "saldomovimentação","Grupo"]).sum().to_dict()["graudeinstrução"]

sectors_stats




{('2023-01-01', -1, 'Agropecuária'): 617,
 ('2023-01-01', -1, 'Comércio'): 4399,
 ('2023-01-01', -1, 'Construção'): 2179,
 ('2023-01-01', -1, 'Indústria'): 2835,
 ('2023-01-01', -1, 'Serviços'): 6221,
 ('2023-01-01', 1, 'Agropecuária'): 290,
 ('2023-01-01', 1, 'Comércio'): 3858,
 ('2023-01-01', 1, 'Construção'): 2417,
 ('2023-01-01', 1, 'Indústria'): 1427,
 ('2023-01-01', 1, 'Serviços'): 6536,
 ('2023-02-01', -1, 'Agropecuária'): 439,
 ('2023-02-01', -1, 'Comércio'): 3835,
 ('2023-02-01', -1, 'Construção'): 2373,
 ('2023-02-01', -1, 'Indústria'): 2211,
 ('2023-02-01', -1, 'Serviços'): 5628,
 ('2023-02-01', 1, 'Agropecuária'): 160,
 ('2023-02-01', 1, 'Comércio'): 3695,
 ('2023-02-01', 1, 'Construção'): 2897,
 ('2023-02-01', 1, 'Indústria'): 1840,
 ('2023-02-01', 1, 'Serviços'): 6408,
 ('2023-03-01', -1, 'Agropecuária'): 1196,
 ('2023-03-01', -1, 'Comércio'): 4017,
 ('2023-03-01', -1, 'Construção'): 2563,
 ('2023-03-01', -1, 'Indústria'): 4067,
 ('2023-03-01', -1, 'Serviços'): 6198,
 ('2

In [77]:
# Novo dicionário com as chaves da segunda e terceira tupla, e os valores associados
new_dict = {(key[1], key[2]): val for key, val in sectors_stats.items()}

# Exibir o novo dicionário
new_dict

# Novo dicionário com as chaves da segunda e terceira tupla, e os valores associados
new_dict = {(key[1], key[2]): val for key, val in sectors_stats.items()}

# Exibir o novo dicionário
print(new_dict)



{(-1, 'Agropecuária'): 2443,
 (-1, 'Comércio'): 4182,
 (-1, 'Construção'): 2901,
 (-1, 'Indústria'): 2530,
 (-1, 'Serviços'): 6567,
 (1, 'Agropecuária'): 207,
 (1, 'Comércio'): 5006,
 (1, 'Construção'): 3546,
 (1, 'Indústria'): 2002,
 (1, 'Serviços'): 8125,
 (-1, 'Não Identificado'): 2}

In [80]:
sectors_stats = {
    ('2023-01-01', -1, 'Agropecuária'): 617,
    ('2023-01-01', -1, 'Comércio'): 4399,
    ('2023-01-01', -1, 'Construção'): 2179,
    ('2023-01-01', -1, 'Indústria'): 2835,
    ('2023-01-01', -1, 'Serviços'): 6221,
    ('2023-01-01', 1, 'Agropecuária'): 290,
    ('2023-01-01', 1, 'Comércio'): 3858,
    ('2023-01-01', 1, 'Construção'): 2417,
    ('2023-01-01', 1, 'Indústria'): 1427,
    ('2023-01-01', 1, 'Serviços'): 6536,
    ('2023-02-01', -1, 'Agropecuária'): 439,
    ('2023-02-01', -1, 'Comércio'): 3835,
    ('2023-02-01', -1, 'Construção'): 2373,
    ('2023-02-01', -1, 'Indústria'): 2211,
    ('2023-02-01', -1, 'Serviços'): 5628,
    ('2023-02-01', 1, 'Agropecuária'): 160,
    ('2023-02-01', 1, 'Comércio'): 3695,
    ('2023-02-01', 1, 'Construção'): 2897,
    ('2023-02-01', 1, 'Indústria'): 1840,
    ('2023-02-01', 1, 'Serviços'): 6408,
    ('2023-03-01', -1, 'Agropecuária'): 1196,
    ('2023-03-01', -1, 'Comércio'): 4017,
    ('2023-03-01', -1, 'Construção'): 2563,
    ('2023-03-01', -1, 'Indústria'): 4067,
    ('2023-03-01', -1, 'Serviços'): 6198,
    ('2024-03-01', 1, 'Agropecuária'): 207,
    ('2024-03-01', 1, 'Comércio'): 5006,
    ('2024-03-01', 1, 'Construção'): 3546,
    ('2024-03-01', 1, 'Indústria'): 2002,
    ('2024-03-01', 1, 'Serviços'): 8125
}

# Novo dicionário com entradas individuais
new_dict = {}
count = 1  # Contador para assegurar chaves únicas

for key, val in sectors_stats.items():
    new_key = (key[1], key[2], count)
    new_dict[new_key] = val
    count += 1

# Exibir o novo dicionário com formatação adequada
from pprint import pprint
pprint(new_dict)




{(-1, 'Agropecuária', 1): 617,
 (-1, 'Agropecuária', 11): 439,
 (-1, 'Agropecuária', 21): 1196,
 (-1, 'Comércio', 2): 4399,
 (-1, 'Comércio', 12): 3835,
 (-1, 'Comércio', 22): 4017,
 (-1, 'Construção', 3): 2179,
 (-1, 'Construção', 13): 2373,
 (-1, 'Construção', 23): 2563,
 (-1, 'Indústria', 4): 2835,
 (-1, 'Indústria', 14): 2211,
 (-1, 'Indústria', 24): 4067,
 (-1, 'Serviços', 5): 6221,
 (-1, 'Serviços', 15): 5628,
 (-1, 'Serviços', 25): 6198,
 (1, 'Agropecuária', 6): 290,
 (1, 'Agropecuária', 16): 160,
 (1, 'Agropecuária', 26): 207,
 (1, 'Comércio', 7): 3858,
 (1, 'Comércio', 17): 3695,
 (1, 'Comércio', 27): 5006,
 (1, 'Construção', 8): 2417,
 (1, 'Construção', 18): 2897,
 (1, 'Construção', 28): 3546,
 (1, 'Indústria', 9): 1427,
 (1, 'Indústria', 19): 1840,
 (1, 'Indústria', 29): 2002,
 (1, 'Serviços', 10): 6536,
 (1, 'Serviços', 20): 6408,
 (1, 'Serviços', 30): 8125}
